In [1]:
import pygame, sys, random
import numpy as np
from keras.layers import Dense
from keras.models import Sequential
from keras.optimizers import Adam
import math
import matplotlib.pyplot as plt
import pylab
from fractions import Fraction

num_episodes = 10

obstacleRadius = 10
agentRadius = 10

# get size of state and action from environment100

boundaryPos = [100, 100]
boundaryLength = [70,70]
boundaryRadius = 40
dispSize = [1280, 960]
initPosAgentStandard = [dispSize[0] / 2, dispSize[1] / 2]
initPosAgent = initPosAgentStandard#[boundaryPos[0] + boundaryLength[0] / 2, boundaryPos[1] + boundaryLength[1] / 2]
goalPos = [50, 50]
goalAngle = 0#random.randrange(0, 360) * math.pi / 180

moveObstacles = False
action_size = 9
obsNumber = 50
state_size = 2
# state_size = obsNumber + 1



Using TensorFlow backend.


In [2]:
# A2C(Advantage Actor-Critic) agent
class A2CAgent:
    def __init__(self, state_size, action_size):
        self.load_model = True
        
        # get size of state and action
        self.state_size = state_size
        self.action_size = action_size
        self.value_size = 1

        # These are hyper parameters for the Policy Gradient
        self.discount_factor = 0.99
        self.actor_lr = 0.00002
        self.critic_lr = 0.00005

        # create model for policy network
        self.actor = self.build_actor()
        self.critic = self.build_critic()

        if self.load_model:
            self.actor.load_weights("./Practice004_DataSave/Actor_Rev.h5")
            self.critic.load_weights("./Practice004_DataSave/Critic_Rev.h5")

    # approximate policy and value using Neural Network
    # actor: state is input and probability of each action is output of model
    def build_actor(self):
        actor = Sequential()
        actor.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        actor.add(Dense(self.action_size, activation='softmax', kernel_initializer='glorot_normal'))
        actor.summary()
        # See note regarding crossentropy in cartpole_reinforce.py
        actor.compile(loss='categorical_crossentropy', optimizer=Adam(lr=self.actor_lr))
        return actor

    # critic: state is input and value of state is output of model
    def build_critic(self):
        critic = Sequential()
        critic.add(Dense(128, input_dim=self.state_size, activation='relu', kernel_initializer='glorot_normal'))
        critic.add(Dense(self.value_size, activation='linear', kernel_initializer='glorot_normal'))
        critic.summary()
        critic.compile(loss="mse", optimizer=Adam(lr=self.critic_lr))
        return critic

    # using the output of policy network, pick action stochastically
    def get_action(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
#         print policy
        return policy
#         return np.random.choice(self.action_size, 1, p=policy)[0]

    # update policy network every episode
    def train_model(self, state, action, reward, next_state, done):
        target = np.zeros((1, self.value_size))
        advantages = np.zeros((1, self.action_size))

        value = self.critic.predict(state)[0]
        next_value = self.critic.predict(next_state)[0]

        if done:
            advantages[0][action] = reward - value
            target[0][0] = reward
        else:
            advantages[0][action] = reward + self.discount_factor * (next_value) - value
            target[0][0] = reward + self.discount_factor * next_value

        self.actor.fit(state, advantages, epochs=1, verbose=0)
        self.critic.fit(state, target, epochs=1, verbose=0)

In [3]:
def ckWall(xPos, yPos):
#     flagWall = 0
#     if(xPos < boundaryPos[0]):
#         xPos = boundaryPos[0]
#         flagWall = -1
#     elif(xPos > boundaryPos[0] + boundaryLength[0]):
#         xPos = boundaryPos[0] + boundaryLength[0]
#         flagWall = -1
#     if(yPos < boundaryPos[1]):
#         yPos = boundaryPos[1]
#         flagWall = -1
#     elif(yPos > boundaryPos[1] + boundaryLeflagWallngth[1]):
#         yPos = boundaryPos[1] + boundaryLength[1]
#         flagWall = -1
    if math.sqrt((xPos - initPosAgent[0]) ** 2 + (yPos - initPosAgent[1]) ** 2) > boundaryRadius - obstacleRadius:
        if xPos - initPosAgent[0] != 0:
            xPos = xPos - (xPos - initPosAgent[0])/abs(xPos - initPosAgent[0])
        if yPos - initPosAgent[1] != 0:
            yPos = yPos - (yPos - initPosAgent[1])/abs(yPos - initPosAgent[1])
    return [int(round(xPos)), int(round(yPos))]

In [4]:
def stateGenerator(obsPosition, agtPosition, idx):
    returnSum = []
    if idx != -1:
        returnSum = returnSum + [agtPosition[0] - obsPosition[idx][0], agtPosition[1] - obsPosition[idx][1]]
    else:
        returnSum = returnSum + [agtPosition[0] - obsPosition[0], agtPosition[1] - obsPosition[1]]
    returnSum = np.reshape(returnSum, [1, 2])
    return returnSum

In [5]:
def takeAction(action):
    xAction = 0
    yAction = 0
    if action == 0:
        xAction = 1
    elif action == 1:
        xAction = 1
        yAction = 1
    elif action == 2:
        xAction = 1
        yAction = -1            
    elif action == 3:
        xAction = -1
        yAction = 1
    elif action == 4:
        xAction = -1
    elif action == 5:
        xAction = -1
        yAction = -1
    elif action == 6:
        yAction = -1
    elif action == 7:
        yAction = 1
    elif action == 8:
        xAction = 0
        yAction = 0
        
    return [xAction, yAction]

In [6]:
def rangeFinder(allObsPos, rangeCenter):
    countObs = 0
    rangeObstacle = [[0,0] for _ in range(obsNumber)]
    for i in range(0, obsNumber):
        if math.sqrt((rangeCenter[0] - allObsPos[i][0])**2 + (rangeCenter[1] - allObsPos[i][1])**2) < boundaryRadius:
            rangeObstacle[countObs] = allObsPos[i]
            countObs += 1
            
    return [countObs, rangeObstacle]

In [7]:
def goalFinder(agtPos):
    goalAngle = math.atan(1.0*(goalPos[1]-agtPos[1])/(goalPos[0]-agtPos[0]))
    tmpGoal = [0,0]
    tmpGoal[0] = int(math.floor(agtPos[0] - boundaryRadius * math.cos(goalAngle)))
    tmpGoal[1] = int(math.floor(agtPos[1] - boundaryRadius * math.sin(goalAngle)))
    return tmpGoal

In [26]:
pygame.init()
screen = pygame.display.set_mode(dispSize)
screen.fill([200, 200, 200])

# make A2C agent
agent = A2CAgent(state_size, action_size)

rList, episodes = [], []

# Make Obstacles (obsNumber)
obstaclePos = [[0, 0] for _ in range(obsNumber)]
for i in range(0,obsNumber):
    obsRadius = random.randrange(agentRadius + 2, 300)
    obsAngle = random.randrange(0,360) * math.pi / 180
    obstaclePos[i][0] = int(initPosAgent[0] + obsRadius * math.cos(obsAngle)) #boundaryPos[0] + random.randrange(1, boundaryLength[0])
    obstaclePos[i][1] = int(initPosAgent[1] + obsRadius * math.sin(obsAngle)) #boundaryPos[1] + random.randrange(1, boundaryLength[1])

for e in range(num_episodes):
    # Initialize
    done = False
    score = 0
    x = initPosAgent[0]
    y = initPosAgent[1]
    print("Episode ", e, "Starts!")
    
    while not done:
        [rangeObsNumber, rangeObsPos] = rangeFinder(obstaclePos, initPosAgent)
#         print rangeObsNumber
        tmpAction = []
        for i in range(0,rangeObsNumber):
            state = stateGenerator(rangeObsPos, [x,y], i)
            policyArr = agent.get_action(state)
            if i == 0:
                tmpAction = (1 - policyArr)
            else:
                tmpAction = tmpAction * (1 - policyArr)
        if rangeObsNumber == 0:
            tmpAction = [1.0/9.0 for _ in range(0, 9)]
        
        tmpGoalPos = goalFinder([x, y])
        state = stateGenerator(tmpGoalPos, [x,y], -1)
        policyArr = agent.get_action(state)
        
        tmpAction = tmpAction * policyArr
        
        tmpAction = tmpAction / np.sum(tmpAction)
        print  tmpAction
        
        action = np.random.choice(action_size, 1, p = tmpAction)[0]
        
        xMove = 0
        yMove = 0

        [xMove, yMove] = takeAction(action)

        x = x + xMove
        y = y + yMove

        wallFlag = 0
        collisionFlag = 0
#         [x, y] = ckWall(x, y)
        pygame.draw.circle(screen, [100, 100, 255], [x,y], 10, 0)
#         next_state = stateGenerator(obstaclePos, [x,y])
        initPosAgent = [x,y]
#         if(math.sqrt((x - initPosAgent[0])**2 + (y - initPosAgent[1])**2) >= agentRadius):
#             initPosAgent = [x, y]
#             obstaclePos = [[0, 0] for _ in range(obsNumber)]
#             for i in range(0,obsNumber):
#                 obsRadius = random.randrange(agentRadius + obstacleRadius + 1, boundaryRadius)
#                 obsAngle = random.randrange(0,360) * math.pi / 180
#                 obstaclePos[i][0] = int(initPosAgent[0] + obsRadius * math.cos(obsAngle))
#                 obstaclePos[i][1] = int(initPosAgent[1] + obsRadius * math.sin(obsAngle))
#             goalArc = math.atan2(y - initPosAgent[1], x - initPosAgent[0]) * 180 / math.pi
#             if goalArc > -30 and goalArc < 30:
        if math.sqrt((x -  goalPos[0])**2 + (y - goalPos[1])**2) <= agentRadius:
            print("Goal Reached!")
            collisionFlag = 1
            done = True
        for i in range(0,obsNumber):
            if moveObstacles:
                obstaclePos[i][0] = obstaclePos[i][0] + random.randrange(-1,1)
                obstaclePos[i][1] = obstaclePos[i][1] + random.randrange(-1,1)
                [obstaclePos[i][0], obstaclePos[i][1]] = ckWall(obstaclePos[i][0], obstaclePos[i][1])
                
            pygame.draw.circle(screen, [255, 50, 50], obstaclePos[i], 10, 0)
            if math.sqrt((x - obstaclePos[i][0])**2 + (y - obstaclePos[i][1])**2) <= 20:
                print("Collision!")
                collisionFlag = -1
                ObjectIndex = i
                done = True
                break
        
        if not done:
            reward = 0.1

        else:
            if collisionFlag == 1:
                reward = 10000 * math.cos(math.atan2(y - initPosAgent[1], x - initPosAgent[0]))
                rList.append(1)
            elif collisionFlag == -1:
                reward = -10000
                rList.append(0)
        
#         agent.train_model(state, action, reward, next_state, done)
        
        score += reward
#         state = next_state
#         if score >= 10000:
#             print "Success!"
#             done = True
        if done:
            # every episode, plot the play time
            initPosAgent = initPosAgentStandard
            obstaclePos = [[0, 0] for _ in range(obsNumber)]
            for i in range(0,obsNumber):
                obsRadius = random.randrange(agentRadius + obstacleRadius + 1, 300)
                obsAngle = random.randrange(0,360) * math.pi / 180
                obstaclePos[i][0] = int(initPosAgent[0] + obsRadius * math.cos(obsAngle))
                obstaclePos[i][1] = int(initPosAgent[1] + obsRadius * math.sin(obsAngle))

            episodes.append(e)
#             pylab.plot(episodes, rList, 'b')
            
        #circle(Surface, color, pos, radius, width=0)
        pygame.draw.circle(screen, [100,255,100], goalPos, 10, 2)
        pygame.draw.circle(screen, [0, 255, 200], tmpGoalPos, 5, 5)
        #rect(Surface, color, Rect, width=0)
#         pygame.draw.rect(screen, [255,100,100],[boundaryPos[0] - agentRadius, boundaryPos[1] - agentRadius, boundaryLength[0] + agentRadius * 2, boundaryLength[1] + agentRadius * 2],2)
        pygame.draw.circle(screen, [255,100,100], initPosAgent, boundaryRadius, 2)
        pygame.display.flip()
        screen.fill([220,220,220])
    print score
    # save the model
#     if e % 50 == 0:
#         agent.actor.save_weights("./Practice004_DataSave/Actor_Rev.h5")
#         agent.critic.save_weights("./Practice004_DataSave/Critic_Rev.h5")
#         pylab.savefig("./Practice004_DataSave/ActorCriticGraph.png")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 128)               384       
_________________________________________________________________
dense_74 (Dense)             (None, 9)                 1161      
Total params: 1,545
Trainable params: 1,545
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 128)               384       
_________________________________________________________________
dense_76 (Dense)             (None, 1)                 129       
Total params: 513
Trainable params: 513
Non-trainable params: 0
_________________________________________________________________
('Episode ', 0, 'Starts!')
[  6.01387755e-29   5.66133375e-25   1.38237539e-

[  5.39417417e-25   5.25048855e-21   1.23992774e-10   2.50523685e-23
   6.57745779e-01   3.42254281e-01   1.08462934e-27   4.56478320e-14
   1.34517997e-28]
[  4.45941245e-25   4.34062623e-21   1.02505941e-10   2.07110193e-23
   5.43751657e-01   4.56248313e-01   8.96672826e-28   3.77374760e-14
   1.11207251e-28]
[  4.94953223e-25   4.81769053e-21   1.13772040e-10   2.29873011e-23
   6.03521645e-01   3.96478325e-01   9.95223336e-28   4.18850779e-14
   1.23429684e-28]
[  5.41153651e-25   5.26738840e-21   1.24391872e-10   2.51330059e-23
   6.59863055e-01   3.40136915e-01   1.08812047e-27   4.57947618e-14
   1.34950983e-28]
[  4.47523898e-25   4.35603166e-21   1.02869747e-10   2.07845253e-23
   5.45681775e-01   4.54318225e-01   8.99855233e-28   3.78714089e-14
   1.11601935e-28]
[  3.50252763e-25   3.40923015e-21   8.05105843e-11   1.62669224e-23
   4.27059621e-01   5.72940409e-01   7.04267995e-28   2.96399021e-14
   8.73447929e-29]
[  2.59181951e-25   2.52278091e-21   5.95766630e-11   1.20

[  6.99504116e-25   6.80871265e-21   1.60788952e-10   3.24873368e-23
   8.52982104e-01   1.47017851e-01   1.40652243e-27   5.91950601e-14
   1.74439853e-28]
[  8.20070757e-25   7.98226383e-21   1.88500160e-10   3.80868608e-23
   9.99998808e-01   1.19732761e-06   1.64895098e-27   6.93979311e-14
   2.04506340e-28]
[  8.20069623e-25   7.98225333e-21   1.88495095e-10   3.80868072e-23
   9.99998093e-01   1.90366495e-06   1.64894867e-27   6.93978363e-14
   2.04506051e-28]
[  8.20068243e-25   7.98223960e-21   1.88485075e-10   3.80867441e-23
   9.99997020e-01   2.99360067e-06   1.64894578e-27   6.93977211e-14
   2.04505714e-28]
[  8.20066468e-25   7.98222183e-21   1.88465063e-10   3.80866589e-23
   9.99995351e-01   4.69605038e-06   1.64894212e-27   6.93975652e-14
   2.04505280e-28]
[  8.20063855e-25   7.98219678e-21   1.88424637e-10   3.80865390e-23
   9.99992549e-01   7.47010199e-06   1.64893692e-27   6.93973484e-14
   2.04504630e-28]
[  8.20059713e-25   7.98215639e-21   1.88343272e-10   3.80

[  6.01208523e-29   5.85194079e-25   1.38196348e-14   2.79221593e-27
   2.53564685e-05   9.99974668e-01   1.20887533e-31   3.32800403e-18
   1.49927266e-32]
[  6.01194801e-29   5.85180717e-25   1.38193188e-14   2.79215218e-27
   4.81596326e-05   9.99951839e-01   1.20884783e-31   1.74539866e-18
   1.49923843e-32]
[  6.83689526e-25   6.65477991e-21   1.57152208e-10   3.17528537e-23
   7.41079569e-01   2.58920401e-01   1.37472340e-27   6.42749395e-15
   1.70496078e-28]
[  4.73031764e-25   4.60431549e-21   1.08727000e-10   2.19691956e-23
   5.59179187e-01   4.40820843e-01   9.51144962e-28   1.22420475e-15
   1.17962992e-28]
[  3.24591561e-25   3.15945351e-21   7.46035525e-11   1.50751303e-23
   3.89230400e-01   6.10769570e-01   6.52669925e-28   4.56614974e-16
   8.09454934e-29]
[  1.99680355e-25   1.94361433e-21   4.58888830e-11   9.27383066e-24
   2.41216123e-01   7.58783877e-01   4.01505693e-28   1.57951276e-16
   4.97955808e-29]
[  9.42773523e-26   9.17660661e-22   2.16587564e-11   4.37

In [ ]:
print("Percent of successful episodes: " + str(100.0 * sum(rList)/num_episodes) + "%")

# plt.bar(range(len(rList)), rList, color = "Blue", width = 0.00001)
# plt.show()